In [1]:
import pandas as pd

In [12]:
df = pd.read_pickle('../data/wss_n_c1')
df_NA = pd.read_pickle('../data/na_ratio_wss_sensor')
df_m = pd.read_pickle('../data/manufacturer')

In [3]:
def cal_speed(df, df_NA):
    sensor = ['C', 'D', 'E', 'F']
    for i, s in enumerate(sensor):
        df_s = df[df.sensor == s].groupby('itapudid').describe().reset_index()
        df_s1 = df_s.itapudid
        df_s2 = df_s.speed
        df_s = pd.concat([df_s1, df_s2], axis=1)
        df_s = df_s[['itapudid', 'mean', 'max', 'std']]
        # Skewness and kurtosis
        df_k = df[df.sensor == s].groupby('itapudid').apply(pd.DataFrame.kurt).reset_index()
        df_k = df_k.rename(columns={'speed':'kurt'})
        df_sk = df[df.sensor == s].groupby('itapudid').skew().reset_index()
        df_sk = df_sk.rename(columns={'speed':'skew'})
        df_k_sk = pd.merge(df_k, df_sk, how='left', on='itapudid')
    
        df_s = pd.merge(df_s, df_k_sk, how='left', on='itapudid')
        df_s['sensor'] = s
        if(s == 'C' or s == 'E'): 
            df_NA_C = df_NA[df_NA.sensor1 == s]
            df_NA_C = df_NA_C[['itapudid', 'na_value_ratio1']].reset_index(drop = True)
            df_NA_C = df_NA_C.rename(columns={'na_value_ratio1':'na_value_ratio'})
        else:
            df_NA_C = df_NA[df_NA.sensor2 == s]
            df_NA_C = df_NA_C[['itapudid', 'na_value_ratio2']].reset_index(drop = True)
            df_NA_C = df_NA_C.rename(columns={'na_value_ratio2':'na_value_ratio'})
        df_m = pd.merge(df_s, df_NA_C, how='left', on='itapudid')
        if(i==0):
            df_f = df_m
        else:
            df_f = pd.concat([df_f, df_m])
    df_f = df_f.sort_values(['itapudid']).reset_index(drop = True)
    
    return df_f

In [4]:
result = cal_speed(df, df_NA)

In [8]:
result = result.dropna(axis=0, how='any').reset_index(drop = True)

In [13]:
result = pd.merge(result, df_m, how='left', on='itapudid')

In [15]:
result = result.dropna(axis=0, how='any').reset_index(drop = True)

In [16]:
result.to_pickle('../data/wss_s')